In [2]:
from ctypes import CDLL
randlib = CDLL("libc.so.6")
import time, sys
import multiprocessing as mp

# Look here for spoilers of what I used to speed up code
import numpy as np
from numba import jit, cuda

def genrand3dpt(MIN, MAX):
    X = (MAX-MIN)*(float(randlib.rand())/2147483647)+MIN
    Y = (MAX-MIN)*(float(randlib.rand())/2147483647)+MIN
    Z = (MAX-MIN)*(float(randlib.rand())/2147483647)+MIN
    return X,Y,Z

class point:
    def __init__(self, x:float ,y: float, z: float):
        self.x, self.y, self.z = x, y, z
    
def setCase(case):
    if(case=='T1'):
        seed = 7
        pts = 100
        expected = "Expected Min, Max: 526.986 15183.808\n"
    elif case=='T2':
        seed = 7
        pts = 1000
        expected = "Expected Min, Max: 70.299 15784.777\n"
    elif case=='T3':
        seed = 7
        pts = 10000
        expected = "Expected Min, Max: 9.730 16509.943\n"
    elif case=='T4':
        seed = 7
        pts = 30000
        expected = "Expected Min, Max:9.270 16643.182\n"
    elif case=='T5':
        seed = 7
        pts = 40005
        expected = "Expected Min, Max: 8.705 16830.027\n"
    return seed, pts, expected

# Parallelization, Calling C and Other Methods: Part 2 of the Hit Series "Speeding Up My Code"

I had never done parallel coding in Python before this project, but I had done it in C in Dr. Butler's class. I needed to do some reading 

In [4]:
%%time
seed, pts, expected = setCase('T1')
randlib.srand(seed)
print(expected)
points = np.empty((pts,3), dtype=np.double)
for i in range(pts):
    points[i]  = genrand3dpt(0, 10000) # call random points macro   


Expected Min, Max: 526.986 15183.808

CPU times: user 2.75 ms, sys: 379 µs, total: 3.13 ms
Wall time: 1.58 ms


In [ ]:


@jit
def shortest_numba(points, ptcnt):
    loopMAX = 0
    loopMIN = 1.7976931348623157e+308 # kinda big
    for i in range(len(points)):
        for j in range(i+1, ptcnt):
            distance = (points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2 + (points[i][2]- points[j][2])**2
            if loopMAX < distance:
                loopMAX = distance
            elif loopMIN > distance:
                loopMIN = distance
          
    print(loopMIN**0.5, loopMAX**0.5)